## 机器学习模型训练流程

### 1. 数据的导入与显示


In [1]:
#检查python版本代码
#sys是系统学习
#sys.version_info  sys.version_info(2, 4, 3, 'final', 0) 
#'final'表示最终,也有'candidate'表示候选，表示版本级别，是否有后继的发行
#major表示主版本号，minor表示次版本号
from sys import version_info
if version_info.major != 2 and version_info.minor != 7:
    raise Exception('请使用Python 2.7来完成此项目')
else:
    print version_info

sys.version_info(major=2, minor=7, micro=12, releaselevel='final', serial=0)


In [2]:
import numpy as np
import pandas as pd

# 数据可视化代码
from titanic_visualizations import survival_stats
from IPython.display import display
%matplotlib inline
#用户代码在Kernel中的shell对象中执行之后，通过DisplayFormatter将需要显示的对象转换为一个描述显示信息的字典，
#并传递给客户端进行显示。目前IPython有三种客户端：控制台、QtConsole和Notebook。
#每种客户端都会从这个字典中提取自己能处理的信息进行显示。

# 加载数据集
in_file = 'titanic_data.csv'
full_data = pd.read_csv(in_file) #读取csv文件

# 显示数据列表中的前几项乘客数据
#a.head(6)表示显示前6行数据，若head()中不带参数则会显示全部数据。
#a.tail(6)表示显示后6行数据，若tail()中不带参数则也会显示全部数据。
display(full_data.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
## 分离结果(标签)与特征
outcomes = full_data['Survived'] # 即labes
data = full_data.drop('Survived',axis=1) #即features

### 2. 分析数据，计算统计信息

In [ ]:
#此处以波士顿房价预测为例
#目标：计算价值的最小值
minimum_price = np.min(prices)

#目标：计算价值的最大值
maximum_price = np.max(prices)

#目标：计算价值的平均值
mean_price = np.mean(prices)

#目标：计算价值的中值
median_price = np.median(prices)

#目标：计算价值的标准差
std_price = np.std(prices)

#目标：输出计算的结果
print "Statistics for Boston housing dataset:\n"
print "Minimum price: ${:,.2f}".format(minimum_price)
print "Maximum price: ${:,.2f}".format(maximum_price)
print "Mean price: ${:,.2f}".format(mean_price)
print "Median price ${:,.2f}".format(median_price)
print "Standard deviation of prices: ${:,.2f}".format(std_price)

### 3. 拆分数据，创造训练集与测试集

In [4]:
from sklearn.model_selection  import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, outcomes, test_size = 0.2, 
                                                    random_state =0)

### 4.定义模型衡量标准

(1)[*决定系数*](http://stattrek.com/statistics/dictionary.aspx?definition=coefficient_of_determination) R<sup>2</sup> 用于量化模型的表现。它是回归分析中十分常用的统计信息，经常被当作衡量模型预测能力好坏的标准。

R<sup>2</sup>的数值范围从0至1，表示**目标变量**的预测值和实际值之间的相关程度平方的百分比。一个模型的R<sup>2</sup> 值为0还不如直接用**平均值**来预测效果好；而一个R<sup>2</sup> 值为1的模型则可以对目标变量进行完美的预测。从0至1之间的数值，则表示该模型中目标变量中有百分之多少能够用**特征**来解释。_模型也可能出现负值的R<sup>2</sup>，这种情况下模型所做预测有时会比直接计算目标变量的平均值差很多。_


In [7]:
## 用sklearn计算决定系数
def performance_metric(y_true, y_predict):
    """计算并返回预测值相比于预测值的分数"""
    from sklearn.metrics import r2_score
    score = r2_score(y_true, y_predict)
    return score

In [ ]:
## 根据公式自己计算决定系数
def performance_metric2(y_true, y_predict):
    """计算并返回预测值相比于预测值的分数"""
    yse = np.array(y_true)-np.array(y_predict)
    sse = np.dot(yse, yse)
    y_mean = np.mean(y_true)
    yst = np.array(y_true)-np.array(y_mean)
    sst = np.dot(yst, yst)   
    score = 1-sse/sst
    return score

#### 学习曲线的书写

### 5.转换非数值特征：独热编码

### 6.特征归一化

### 7. 模型训练

#### 交叉验证
1. 在K折交叉验证中，用到的数据是训练集中的所有数据，过程是将训练集的所有数据平均划分成K份，取第K份作为验证集，余下的K-1份作为交叉验证的训练集。Kfold默认是把数据按顺序切分成K份，而当设置shuffle = True时，是将数据打乱后再切分。
2. GridSearchCV是将参数构成网格状的组合，每一次交叉验证选取其中的一组参数，直至所有参数全部被验证，比较所有参数组合结果，选取模型最优时的参数组合。

In [ ]:
# 导入依赖包
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
import numnpy as np
import pandas as pd

# 设置参数
params={'learning_rate':np.linspace(0.05,0.25,5), 
        'max_depth':[x for x in range(1,8,1)], 
        'min_samples_leaf':[x for x in range(1,5,1)],
        'n_estimators':[x for x in range(50,100,10)]}

#设置模型和评价指标，训练
clf = GradientBoostingClassifier()
grid = GridSearchCV(clf, params, cv=10, scoring="f1")#f1为评价标准函数
grid.fit(X, y)

# 查看最佳分数
grid.best_score_    #查看最佳分数(此处为f1_score)
grid.best_params_   #查看最佳参数

# 获取最佳分数
grid.best_estimator_

#用最佳分数预测
best_model=grid.best_estimator_
predict_y=best_model.predict(Test_X)
metrics.f1_score(y, predict_y)

In [ ]:
### 例子：交叉验证选择最优模型
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

def fit_model(X,y):
 
    cross_validator = KFold(10) #创建交叉验证生成器

    regressor = DecisionTreeRegressor() #创建决策树回归函数

    params = {"max_depth": range(1,11)}

    scoring_fnc = make_scorer(performance_metric)

    grid = GridSearchCV(regressor, params, scoring_fnc, cv=cross_validator)
    
    # 基于输入数据 [X,y],执行拟合和预测数据的试验。
    grid = grid.fit(X, y)
    #print pd.DataFrame(grid.cv_results_)
    # 返回网格搜索后的最优模型
    return grid.best_estimator_

### 8.模型调整及评价